In [2]:
import pandas as pd
import numpy as np
import time
import os
import datetime as dt
from functools import reduce
import unidecode #para tirar os acentos e "ç"
import re

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [3]:
df0 = pd.read_csv('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//AAPredictions//BetanoOdds//Filtered_Data//Betano0.csv')
df1 = pd.read_csv('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//AAPredictions//BetanoOdds//Filtered_Data//Betano1.csv')

In [20]:
leagues_betano = {
    'Championship':'Championship', 'LaLiga':'La_Liga',
    'Liga Europa - Jogos':'Liga_Europa','Liga dos Campeões - Jogos':'Champions',
    'MLS':'MLS','Premier League' : 'Premier_League',
    'Premier Liga':'RFPL','Premiership':'Premiership',
    'Primeira Liga':'Primeira_Liga','Série A': 'Serie_A',
    'Bundesliga':'Bundesliga','Ligue 1':'Ligue_1'
}

In [21]:
def Betano_date(x):
    months = {
        'Janeiro':1,'Fevereiro':2,'Março':3,'Abril':4,'Maio':5,'Junho':6,
        'Julho':7,'Agosto':8,'Setembro':9,'Outubro':10,'Novembro':11,'Dezembro':12
    }
    try:
        x = x.split(', ')[1]
        day,month,year,time=x.split(' ')
        #date
        if month not in months:
            print(month)
            month = input('Qual é o mês?   ')
        else:
            month = months[month]
        date = dt.datetime.strptime(str(day)+'/'+str(month)+'/'+str(year),'%d/%m/%Y')
        #time
        h,m=time.split(':')
        H=int(h)*100
        time = H+int(m)
        return date,time
    except:
        return 'None','None'

def Betano_league(x,leagues):
    if x not in leagues:
        print(x)
        return 'None'
    else:
        return leagues[x]
    
def Betano_nan(df): #remove os scrapes falhados
    lig=list(np.where(df['League'].isnull().values)[0])
    rest=list(np.where(np.logical_and(df['Date'].isnull().values,df['HomeTeam'].isnull().values,
                                     df['AwayTeam'].isnull().values))[0])
    dele = list(np.unique(np.array(lig+rest)))
    print(dele)
    df=df.drop(dele,axis=0).reset_index(drop=True)
    return df

In [22]:
def europe_filter(df):
    for i in range(len(df)):
        if (df.iloc[i]['League']=='Liga_Europa') or (df.iloc[i]['League']=='Champions'):
            for c in list(df.columns[8:-2]):
                df.at[i,c]='None'
    return df
            

In [23]:
def Betano_filter(df,l=leagues_betano):
    df = Betano_nan(df)
    Dates = [df.iloc[i]['Date'] for i in range(len(df))]
    date_time = list(map(Betano_date,Dates))
    
    try:
        sDates = [df.iloc[i]['Current_Date'] for i in range(len(df))]
        sdate_time = list(map(Betano_date,sDates))
    except:
        pass
    
    Leag = [df.iloc[i]['League'] for i in range(len(df))]
    League = list(map(Betano_league,Leag,[l]*len(Leag)))
    Y25=[]
    N25=[]
    YBS=[]
    NBS=[]
    X1=[]
    _12=[]
    X2=[]
    for i in range(len(df)):
        #25
        if df.iloc[i]['Tag25']=='Mais de2.5':
            Y25+=[df.iloc[i]['ODD_25+']]
            N25+=[df.iloc[i]['ODD_25-']]
        elif df.iloc[i]['Tag15']=='Mais de2.5':
            Y25+=[df.iloc[i]['ODD_15+']]
            N25+=[df.iloc[i]['ODD_15-']]
        elif df.iloc[i]['Tag35']=='Mais de2.5':
            Y25+=[df.iloc[i]['ODD_35+']]
            N25+=[df.iloc[i]['ODD_35-']]
        else:
            if (df.iloc[i]['League']=='Liga Europa - Jogos') or (df.iloc[i]['League']=='Liga dos Campeões - Jogos'):
                Y25+=['None']
                N25+=['None']
            else:
                print(df.iloc[i]['League'],i)
                print('problema com 25')
                print('---------------')
                Y25+=['None']
                N25+=['None']
            
        #BS
        if df.iloc[i]['TagBS']=='Ambas Equipas Marcam (Sim/Não)':
            YBS+=[df.iloc[i]['ODD_BthScore_Yes']]
            NBS+=[df.iloc[i]['ODD_BthScore_No']]
        else:
            if (df.iloc[i]['League']=='Liga Europa - Jogos') or (df.iloc[i]['League']=='Liga dos Campeões - Jogos'):
                YBS+=['None']
                NBS+=['None']
            else:
                print(df.iloc[i]['League'],i)
                print('problema com BS')
                print('---------------')
                YBS+=['None']
                NBS+=['None']
            
        #DC
        if df.iloc[i]['Tag1X']=='1X':
            X1+=[df.iloc[i]['ODD_1X']]
        elif df.iloc[i]['Tag12']=='1X':
            X1+=[df.iloc[i]['ODD_12']]
        elif df.iloc[i]['TagX2']=='1X':
            X1+=[df.iloc[i]['ODD_X2']]
        else:
            X1+=['None']
        #--------------------------------------------------   
        if df.iloc[i]['Tag1X']=='12':
            _12+=[df.iloc[i]['ODD_1X']]
        elif df.iloc[i]['Tag12']=='12':
            _12+=[df.iloc[i]['ODD_12']]
        elif df.iloc[i]['TagX2']=='12':
            _12+=[df.iloc[i]['ODD_X2']]
        else:
            _12+=['None']    
        #---------------------------------------------------
        if df.iloc[i]['Tag1X']=='2X':
            X2+=[df.iloc[i]['ODD_1X']]
        elif df.iloc[i]['Tag12']=='2X':
            X2+=[df.iloc[i]['ODD_12']]
        elif df.iloc[i]['TagX2']=='2X':
            X2+=[df.iloc[i]['ODD_X2']]
        else:
            X2+=['None']
            
    ODDH = list(df.iloc[:]['ODDH'])
    ODDD = list(df.iloc[:]['ODDD'])
    ODDA = list(df.iloc[:]['ODDA'])
    Date = [date_time[i][0] for i in range(len(date_time))]
    Time = [date_time[i][1] for i in range(len(date_time))]
    
    try:
        sDate = [sdate_time[i][0] for i in range(len(sdate_time))]
        sTime = [sdate_time[i][1] for i in range(len(sdate_time))]
    except:
        pass
    
    HT = list(map(lambda x:unidecode.unidecode(x),list(df.iloc[:]['HomeTeam'])))
    AT = list(map(lambda x:unidecode.unidecode(x),list(df.iloc[:]['AwayTeam'])))
    
    try:
        df = pd.DataFrame({
            'League':League,'Date':Date,'Time':Time,
            'HT':HT,'AT':AT,'ODDH':ODDH,'ODDD':ODDD,'ODDA':ODDA,
            'ODD_25+':Y25,'ODD_25-':N25,
            'ODD_BthScore_Yes':YBS,'ODD_BthScore_No':NBS,
            'ODD_1X':X1,'ODD_12':_12,'ODD_X2':X2,
            'Scrape_Date':sDate,'Scrape_Time':sTime
        })
    except:
        df = pd.DataFrame({
            'League':League,'Date':Date,'Time':Time,
            'HT':HT,'AT':AT,'ODDH':ODDH,'ODDD':ODDD,'ODDA':ODDA,
            'ODD_25+':Y25,'ODD_25-':N25,
            'ODD_BthScore_Yes':YBS,'ODD_BthScore_No':NBS,
            'ODD_1X':X1,'ODD_12':_12,'ODD_X2':X2
        })
    
    df = europe_filter(df)
    
    return df.sort_values(['Date','Time'],ascending=[True,True]).reset_index(drop=True)


def mergeB_year(x):
    x=x.split('-')
    return x[0]



def days_diff(x,y): #absoluto do número de dias de diferença entre duas datas
    if isinstance(x,str):
        if '-' in x:
            x=dt.datetime.strptime(x,'%Y-%m-%d')
        else:
            print(x)
            raise Exception('Weird Date string')
    if isinstance(y,str):
        if '-' in y:
            y=dt.datetime.strptime(y,'%Y-%m-%d')
        else:
            print(y)
            raise Exception('Weird Date string')
    diff = abs((x-y).days)
    return diff



    
def mergeB_month(x): #não vou usar por causa daquele pequeno erro da falhas das datas no último dia do mês
    x=x.split('-')
    return x[1]
    
def merge_Betano(w): #w é uma lista de dfs
#se encontrarmos o mesmo jogo, escolhemos a odd + baixa (da ODDH)
    df = pd.concat(w,sort=False).sort_values(['Date','Time'],ascending=[True,True]).reset_index(drop=True)
    i=0
    while i<len(df):
        same = list(np.where(np.logical_and(
        np.logical_and(df['HT']==df.iloc[i]['HT'],df['AT']==df.iloc[i]['AT']),
        np.logical_and(np.array(list(map(lambda x:mergeB_year(x),df['Date'])))==mergeB_year(df.iloc[i]['Date']),
        df['League']==df.iloc[i]['League'])))[0])
        if len(same)==1:
            i+=1
        else:
            ind=np.argmin([df.iloc[i]['ODDH'] for i in same])
            del same[ind]
            df = df.drop(same,axis=0).reset_index(drop=True)
            #!!!!!!!!!!!!!!!!!!!!!        
            #Se precisar do 25 e do BS, mudar este merge, para tentar ter um q não tenha 'None' nessas odds
            #!!!!!!!!!!!!!!!!!!!!!!
    return df


def merge_Betano(w): #w é uma lista de dfs
#se encontrarmos o mesmo jogo, escolhemos a odd + baixa (da ODDH)
    df = pd.concat(w,sort=False).sort_values(['Date','Time'],ascending=[True,True]).reset_index(drop=True)
    i=0
    while i<len(df):
        same = list(np.where(np.logical_and(
        np.logical_and(df['HT']==df.iloc[i]['HT'],df['AT']==df.iloc[i]['AT']),
        np.logical_and(np.array(list(map(lambda x,y:days_diff(x,y),df['Date'],df.iloc[i]['Date'])))<=10,
        df['League']==df.iloc[i]['League'])))[0])
        if len(same)==1:
            i+=1
        else:
            ind=np.argmin([df.iloc[i]['ODDH'] for i in same])
            del same[ind]
            df = df.drop(same,axis=0).reset_index(drop=True)
            #!!!!!!!!!!!!!!!!!!!!!        
            #Se precisar do 25 e do BS, mudar este merge, para tentar ter um q não tenha 'None' nessas odds
            #!!!!!!!!!!!!!!!!!!!!!!
    return df


def Betano_timediff(df,i): #damos a df e o índice e devolve a diferença entre scrape time e game time (+/- em horas)
    scrape_date = dt.datetime.strptime(df.iloc[i]['Scrape_Date'],'%Y-%m-%d')
    scrape_time = df.iloc[i]['Scrape_Time']
    date = dt.datetime.strptime(df.iloc[i]['Date'],'%Y-%m-%d')
    time = df.iloc[i]['Time']
    sh,sm = str(scrape_time)[0:2],str(scrape_time)[2:4]
    h,m = str(time)[0:2],str(time)[2:4]
    dt_ok = date + dt.timedelta(hours=int(h),minutes=int(m))
    dt_scrape = scrape_date + dt.timedelta(hours=int(sh),minutes=int(sm))
    final = dt_ok-dt_scrape
    return ((final.days*86400)+(final.seconds))//3600


def Betano_odd(w,odd='ODDH',x=3): #devolve uma df com o top "x" de Odds do evento "odd"
    df = pd.concat(w,sort=False).sort_values(['Date','Time'],ascending=[True,True]).reset_index(drop=True)
    for n in range(1,x+1):
        vars()[odd+'_TOP'+str(n)]=[]
        vars()['Time_'+str(n)]=[]
    HT=[]
    AT=[]
    Lig=[]
    Dt=[]
    Tm=[]
    i=0
    done_is=[]
    while i<len(df):
        if i in done_is:
            i+=1
            continue
        same = list(np.where(np.logical_and(
        np.logical_and(df['HT']==df.iloc[i]['HT'],df['AT']==df.iloc[i]['AT']),
        np.logical_and(np.array(list(map(lambda x,y:days_diff(x,y),df['Date'],df.iloc[i]['Date'])))<=10,
        np.array(list(map(lambda x:mergeB_month(x),df['Date'])))==mergeB_month(df.iloc[i]['Date']))))[0])
        if len(same)==1:
            for n in range(1,x+1):
                if n==1:
                    vars()[odd+'_TOP'+str(n)]+=[df.iloc[i]['ODDH']]
                    vars()['Time_'+str(n)]+=[Betano_timediff(df,i)] #diff em horas do game time e scrape time
                else:
                    vars()[odd+'_TOP'+str(n)]+=['None']
                    vars()['Time_'+str(n)]+=['None']
            HT+=[df.iloc[same[0]]['HT']]
            AT+=[df.iloc[same[0]]['AT']]
            Lig+=[df.iloc[same[0]]['League']]
            Dt+=[df.iloc[same[0]]['Date']]
            Tm+=[df.iloc[same[0]]['Time']]
        else:
            oddsH=[df.iloc[s]['ODDH'] for s in same]
            oddsH_=list(map(lambda x:-x,oddsH))
            sorte=np.argsort(oddsH_) #assim fazemos reverse sort
            i2=1
            while (i2<=x):
                if ((i2-1)<len(sorte)):
                    vars()[odd+'_TOP'+str(i2)]+=[oddsH[sorte[i2-1]]]
                    vars()['Time_'+str(i2)]+=[Betano_timediff(df,same[sorte[i2-1]])]
                else:
                    vars()[odd+'_TOP'+str(i2)]+=['None']
                    vars()['Time_'+str(i2)]+=['None']
                i2+=1
            HT+=[df.iloc[same[0]]['HT']]
            AT+=[df.iloc[same[0]]['AT']]
            Lig+=[df.iloc[same[0]]['League']]
            Dt+=[df.iloc[same[0]]['Date']]
            Tm+=[df.iloc[same[0]]['Time']]
                
            
        done_is+=same
        i+=1
    
    odd_df=pd.DataFrame({'League':Lig,'HT':HT,'AT':AT,'Date':Dt,'Time':Tm})
    for n in range(1,x+1):
        odd_df[odd+'_TOP'+str(n)]=vars()[odd+'_TOP'+str(n)]
        odd_df['Time_'+str(n)]=vars()['Time_'+str(n)]
    
    return odd_df


            
def get_int_str(r): #dá os inteiros de uma string, seguidos
    s = ''.join(x for x in r if x.isdigit())
    return int(s)


def save_filterBetano(w):
    oo=os.listdir('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//AAPredictions//BetanoOdds//Filtered_Data')
    if oo!=[]:
        i1 = max(list(map(lambda x:get_int_str(x),oo)))
        w.to_csv('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//AAPredictions//BetanoOdds//Filtered_Data//Betano'+str(i1+1)+'.csv',index=False)            
    else:
        w.to_csv('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//AAPredictions//BetanoOdds//Filtered_Data//Betano0.csv',index=False)
        

def save_rawBetano(b):
    dir1 = os.listdir('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//AAPredictions//BetanoOdds')
    dfs = [dir1[i] for i in range(len(dir1)) if ('betano' in dir1[i]) and ('Pre' not in dir1[i])]
    i1 = max(list(map(lambda x:get_int_str(x),dfs)))
    w.to_csv('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//AAPredictions//BetanoOdds//betano'+str(i1+1)+'.csv',index=False)
    
    
def Betano_autofilter():
#verifica quais as dfs q estão no filter data, e faz o filter das dfs que ainda não foram filtered
    dir1 = os.listdir('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//AAPredictions//BetanoOdds')
    dfs = [dir1[i] for i in range(len(dir1)) if ('betano' in dir1[i]) and ('Pre' not in dir1[i])]
    dir2 = os.listdir('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//AAPredictions//BetanoOdds//Filtered_Data')
    i1 = max(list(map(lambda x:get_int_str(x),dfs)))
    i2 = max(list(map(lambda x:get_int_str(x),dir2)))
    
    for i in range(i2+1,i1+1):
        dff = pd.read_csv('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//AAPredictions//BetanoOdds//betano'+str(i)+'.csv')
        Df = Betano_filter(dff)
        save_filterBetano(Df)
        print('Done ',i)

In [18]:
abs((dt.datetime.strptime('2020/02/01','%Y/%m/%d')-dt.datetime(2020,3,4)).days)

32

In [49]:
Betano_autofilter()

[10, 13, 21]
MLS 3
problema com 25
---------------
MLS 3
problema com BS
---------------
MLS 4
problema com 25
---------------
MLS 4
problema com BS
---------------
Championship 15
problema com 25
---------------
Championship 15
problema com BS
---------------
MLS 28
problema com 25
---------------
MLS 28
problema com BS
---------------
MLS 33
problema com 25
---------------
MLS 33
problema com BS
---------------
Done  9
[0, 22, 41]
MLS 14
problema com 25
---------------
MLS 14
problema com BS
---------------
MLS 19
problema com 25
---------------
MLS 19
problema com BS
---------------
Championship 22
problema com 25
---------------
Championship 22
problema com BS
---------------
MLS 39
problema com 25
---------------
MLS 39
problema com BS
---------------
Done  10
[4, 16, 20, 21, 22]
MLS 19
problema com 25
---------------
MLS 19
problema com BS
---------------
Done  11
[0, 2, 4, 7, 10, 11, 12, 14]
Done  12
[0, 10, 11, 12, 15, 35]
Done  13


In [50]:
df = pd.read_csv('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//AAPredictions//BetanoOdds//Filtered_Data//Betano13.csv')

In [52]:
len(np.where(df['League']=='Ligue_1')[0])

7

In [9]:
df0 = pd.read_csv('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//AAPredictions//BetanoOdds//Filtered_Data//Betano0.csv')
df1 = pd.read_csv('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//AAPredictions//BetanoOdds//Filtered_Data//Betano1.csv')
df2 = pd.read_csv('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//AAPredictions//BetanoOdds//Filtered_Data//Betano2.csv')
df3 = pd.read_csv('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//AAPredictions//BetanoOdds//Filtered_Data//Betano3.csv')
df4 = pd.read_csv('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//AAPredictions//BetanoOdds//Filtered_Data//Betano4.csv')
df5 = pd.read_csv('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//AAPredictions//BetanoOdds//Filtered_Data//Betano5.csv')
df6 = pd.read_csv('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//AAPredictions//BetanoOdds//Filtered_Data//Betano6.csv')
df7 = pd.read_csv('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//AAPredictions//BetanoOdds//Filtered_Data//Betano7.csv')
df8 = pd.read_csv('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//AAPredictions//BetanoOdds//Filtered_Data//Betano8.csv')

In [10]:
Betano_odd([df0,df1,df2,df3,df4,df5,df6,df7,df8])

,League,HT,AT,Date,Time,ODDH_TOP1,Time_1,ODDH_TOP2,Time_2,ODDH_TOP3,Time_3
0,Premier_League,Bournemouth,Southampton,2020-07-19,1400,2.40,2,None,None,None,None
1,Championship,Derby County,Leeds United,2020-07-19,1400,2.98,2,None,None,None,None
2,Championship,Barnsley FC,Nottingham Forest,2020-07-19,1500,2.62,3,None,None,None,None
3,Premier_League,Tottenham,Leicester,2020-07-19,1600,2.46,4,None,None,None,None
4,La_Liga,Alaves,Barcelona,2020-07-19,1600,6.20,4,None,None,None,None
5,Serie_A,Parma,Sampdoria,2020-07-19,1615,3.00,4,None,None,None,None
6,La_Liga,Villarreal,Eibar,2020-07-19,1730,1.70,5,None,None,None,None
7,La_Liga,Real Valladolid,Betis,2020-07-19,1730,2.62,5,None,None,None,None
8,Serie_A,Fiorentina,Torino,2020-07-19,1830,1.88,6,None,None,None,None
9,Serie_A,Napoles,Udinese,2020-07-19,1830,1.60,6,None,None,None,None


In [301]:
df_all = pd.concat([df0,df1,df2],sort=False)

In [303]:
df_all.loc[(df_all['HT']=='CSKA Moscovo') & (df_all['AT']=='Tambov')]

,Unnamed: 0,League,Date,Time,HT,AT,ODDH,ODDD,ODDA,ODD_25+,ODD_25-,ODD_BthScore_Yes,ODD_BthScore_No,ODD_1X,ODD_12,ODD_X2,Scrape_Date,Scrape_Time
39,39,RFPL,2020-07-22,1700,CSKA Moscovo,Tambov,1.28,4.15,6.45,1.65,1.75,1.95,1.5,1.03,1.11,2.98,2020-07-19,1145
26,26,RFPL,2020-07-22,1700,CSKA Moscovo,Tambov,1.22,4.45,7.35,1.55,1.87,1.95,1.5,1.02,1.13,3.30,2020-07-21,1354
42,42,RFPL,2020-07-22,1700,CSKA Moscovo,Tambov,1.26,4.65,5.80,1.53,1.9,1.85,1.57,1.05,1.08,3.05,2020-07-22,1450


In [ ]:
def Betano_autofilter():
#verifica quais as dfs q estão no filter data, e faz o filter das dfs que ainda não foram filtered
    dir1 = os.listdir('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//AAPredictions//BetanoOdds')
    dfs = [dir1[i] for i in range(len(dir1)) if ('betano' in dir1[i]) and ('Pre' not in dir1[i])]
    dir2 = os.listdir('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//AAPredictions//BetanoOdds//Filtered_Data')
    i1 = max(list(map(lambda x:get_int_str(x),dfs)))
    i2 = max(list(map(lambda x:get_int_str(x),dir2)))
    
    for i in range(i1+1,i2+1):
        dff = pd.read_csv('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//AAPredictions//BetanoOdds//betano'+str(i)+'.csv')
        Df = Betano_filter(dff)
        save_filterBetano(Df)

In [38]:
dir1 = os.listdir('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//AAPredictions//BetanoOdds')
dfs = [dir1[i] for i in range(len(dir1)) if ('betano' in dir1[i]) and ('Pre' not in dir1[i])]
dir2 = os.listdir('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//AAPredictions//BetanoOdds//Filtered_Data')
i1 = max(list(map(lambda x:get_int_str(x),dfs)))
i2 = max(list(map(lambda x:get_int_str(x),dir2)))
print(i1,i2)


13 8
